# Additional Notebook for HW3
This notebook was created to retrain the linear model until we are able to find a model,
wherein the r^2 score of the test dataset is better than the train dataset
By: John Markton Olarte

In [1]:
# Import the necessary python packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# NOTE: sklearn should only be used for the following:
#     Standardization and Dividing the Data into Training and Testing Sets
#     Checking MSE and r^2 values
#     Lastly for comparison of implementation (from scratch) and the OLS results since sklearn uses least squares for linear regression.
from sklearn import preprocessing                           # Used for standardization
from sklearn.model_selection import train_test_split        # Used for dividing the dataset into training and testing sets
from sklearn.metrics import r2_score, mean_squared_error    # Used for checking MSE and r^2 values
from sklearn.linear_model import LinearRegression           # Used for the OLS results

%matplotlib inline

In [2]:
# Load the data from Advertising.csv into a pandas dataframe
df = pd.read_csv('Advertising.csv')

# Dataframe Cleaning
# Since the first column is just the index, we can drop it
df = df.drop('Unnamed: 0', axis=1)

# In this process, we will be using the preprocessing.scale function from sklearn
df_standardized = preprocessing.scale(df)

# Convert the standardized data into a dataframe
df_standardized = pd.DataFrame(df_standardized, columns=df.columns)

In [3]:
# We need to add this column as it will serve as our bias term.
# Basically, in the formula y = theta_0 + theta_1 * x_1 + theta_2 * x_2 + ... + theta_n * x_n
#   The theta_0 is the bias term and for us to include it in the computation we need to add a column of 1's.
#   theta_0 * 1 = theta_0 (by Identity Property of Multiplication)
df_standardized.insert(0, 'all_ones', 1)
df_standardized

,all_ones,TV,Radio,Newspaper,Sales
0,1,0.969852,0.981522,1.778945,1.552053
1,1,-1.197376,1.082808,0.669579,-0.696046
2,1,-1.516155,1.528463,1.783549,-0.907406
3,1,0.052050,1.217855,1.286405,0.860330
4,1,0.394182,-0.841614,1.281802,-0.215683
...,...,...,...,...,...
195,1,-1.270941,-1.321031,-0.771217,-1.234053
196,1,-0.617035,-1.240003,-1.033598,-0.830548
197,1,0.349810,-0.942899,-1.111852,-0.234898
198,1,1.594565,1.265121,1.640850,2.205347


In [4]:
# Feature Selection: TV, Radio, Newspaper, and all_ones (bias term)
features = df_standardized[['all_ones', 'TV', 'Radio', 'Newspaper']]
# Select the Sales column as the target (response) variable
response = df_standardized['Sales']

# We will be using 75% of the data for training and 25% for testing
X_train, X_test, y_train, y_test = train_test_split(features, response, train_size=0.75, test_size=0.25)

In [5]:
def initialize_weights(X, random=False):
    # Initialize the length of the weights to the number of columns in X
    len_X = X.shape[1]
    
    # If the random parameter is set to True, then we will initialize the weights randomly
    if random:
        return np.random.rand(len_X)
    # Otherwise, by default, we will initialize the weights to zero
    return np.zeros(len_X)

def predict(X, weights):
    # From the formula we need to multiply each weight to its corresponding feature from X
    #   we can do this using the dot product function from numpy.
    y_hat = np.dot(X, weights)

    return y_hat

def compute_cost(X, y, weights):  
    # Initialize the length of the data
    # The length of the data is equal to the length of either the response or the features
    # For simplicity, we will use the length of the response
    m = len(y)

    # Calculate the cost function
    # From the formula, we need to square the difference between the predicted and actual values and find their sum
    #   as indicated by this formula [sum((y_hat - y)^2)]
    #   and then we need to multiply it by 1/2m.
    cost = 1/(2*m) * np.sum((predict(X, weights) - y)**2)
    
    return cost

def compute_gradient(X, y, weights):
    # Initialize the length of the data
    m = len(y)
    
    # X.T is the transpose of X which we will apply to each difference between the predicted and actual values using the dot product.
    w = 1/m * np.dot(X.T, (predict(X, weights) - y))

    return w

def update_weights(X, y, weights, alpha):
    # Update the weights: theta = theta - alpha * gradient
    #   As mentioned from my notes in compute_gradient, we need to multiply alpha (learning rate) to the gradient
    #   and then subtract it from the weights.
    updated_weights = weights - alpha * compute_gradient(X, y, weights)

    return updated_weights

def gradient_descent(X, y, weights, alpha, iterations):
    # Since we are to produce 2 matrices, we first initialize these two matrices
    weights_history = [0] * iterations
    cost_history = [0] * iterations

    # Loop through the number of iterations
    # NOTE: There are actually two main types of gradient descent, batch gradient descent and stochastic gradient descent.
    #      In our case, we are implementing the batch gradient descent. That is why we are looping through the entire dataset.
    #      If we were to implement the stochastic gradient descent, we would only need to loop through a single data point.
    for i in range(iterations):
        # Update the weights
        weights = update_weights(X, y, weights, alpha)
        # Save the weights in the weights history matrix
        weights_history[i] = weights
        # Compute the cost
        cost = compute_cost(X, y, weights)
        # Save the cost in the cost history matrix
        cost_history[i] = cost
    
    return weights_history, cost_history

def plot_costs(cost_array):
    # Set the size of the plot
    plt.figure(figsize=(10, 6))
    plt.plot(cost_array)
    plt.title('Cost over Iterations')
    plt.xlabel('Iterations')
    plt.ylabel('Cost')
    plt.show()

In [6]:
# Find the optimal weights using the training set (Do this until r^2 score train < test)
r2_score_train = 0
r2_score_test = 0
optimal_weights = []
iteration = 1

while (r2_score_train >= r2_score_test):
    init_w = initialize_weights(X_train, random=True) # We will use random initialization
    print(f"Inital Weights: {init_w}")

    # Set the iterations and the learning rate
    iterations = 50000                          # Let's do 50,000 iterations, as recommended in the activity guide.
    alpha = 0.01                                 # Let's use a learning rate of 0.01

    # Run the gradient descent algorithm
    weights_history, cost_history = gradient_descent(X_train, y_train, init_w, alpha, iterations)

    # Get the final weights (Optimal weights)
    # We will assume that the optimal weights are the weights that give us the lowest cost, 
    #   which can be found at the end of the weights_history array.
    optimal_weights = weights_history[-1]
    print(f"Optimal Weights: {optimal_weights}")

    # calculate r^2 scores
    r2_score_train = r2_score(y_train, predict(X_train, optimal_weights))
    r2_score_test = r2_score(y_test, predict(X_test, optimal_weights))

    print(f"Iteration {iteration} - r^2 score train: {r2_score_train}, r^2 score test: {r2_score_test}\n")
    iteration += 1
    

print("----FINAL----")
# Print the optimal weights
print(f"Optimal Weights: {optimal_weights}")

# Print the r^2 scores
print(f"r^2 score train: {r2_score_train}")
print(f"r^2 score test: {r2_score_test}")

Inital Weights: [0.79230802 0.79409756 0.22644812 0.92839463]
Optimal Weights: [ 0.00487484  0.75809584  0.52960603 -0.02070791]
Iteration 1 - r^2 score train: 0.8899415563505054, r^2 score test: 0.9157629399026007

----FINAL----
Optimal Weights: [ 0.00487484  0.75809584  0.52960603 -0.02070791]
r^2 score train: 0.8899415563505054
r^2 score test: 0.9157629399026007
